In [37]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests as rq
from github import Github, GithubException
import time
from shutil import copyfile

In [39]:
headers = {'user-agent':'Mozilla/5.0'}

credentials = Github("login", "password")

search = credentials.search_code("filename:dockerfile")
print(len(list(search.get_page(0))))
print(search.totalCount)

size = search.totalCount/len(list(search.get_page(0)))
print(size)
page_n = 0
while(page_n<size):
    try:
        results = list(search.get_page(page_n))
        files=[]
        for file in results:
            page = rq.get(file.html_url,headers=headers)
            parsed_page = BeautifulSoup(page.text,"html.parser").find("table",attrs={'class':'highlight tab-size js-file-line-container'})
            dockerfile = ""
            if(parsed_page.find_all("td",attrs={'class':'blob-code blob-code-inner js-file-line'})!=None):
                for td in parsed_page.find_all("td",attrs={'class':'blob-code blob-code-inner js-file-line'}):
                    dockerfile += td.text + "\n"
            page = rq.get(file.html_url,headers=headers)
            code_link = BeautifulSoup(page.text,"html.parser").find("a",attrs={'data-hotkey':'g c'})
            page = rq.get("https://github.com"+code_link.get('href'),headers=headers)
            language = BeautifulSoup(page.text,"html.parser").find("ol",attrs={'class':'repository-lang-stats-numbers'})
            if(language!=None):
                languages = BeautifulSoup(page.text,"html.parser").find("ol",attrs={'class':'repository-lang-stats-numbers'}).text.strip()
            else :
                languages = None           
            repo_name = file.repository.full_name
            repo_date = file.repository.created_at
            repo_last_updated = file.repository.updated_at
            repo_organization = file.repository.organization
            files.append(dict(zip(['Repositório','Organização','Dockerfile','Linguagens','Data Criação','Data Update','Repo link'],\
                                 [repo_name,repo_organization,dockerfile,languages,repo_date,repo_last_updated,"https://github.com"+code_link.get('href')])))
        dataDockers = pd.DataFrame(files,\
                                       columns=['Repositório','Organização','Dockerfile',
                                                'Linguagens','Data Criação','Data Update','Repo link'])
        if(page_n==0):
            dataDockers.to_csv('dockers.csv',mode='a')
        else:
            dataDockers.to_csv('dockers.csv',mode='a',header=False)
        page_n+=1
    except GithubException:
        print("Extourou as requisições por minuto, irá esperar um minuto para continuar")
        copyfile('dockers.csv', 'csv/dockers.csv')
        time.sleep(60)
copyfile('dockers.csv', 'csv/dockers.csv')

gaierror: [Errno -2] Name or service not known